# BLEND019

- Automate the Blend Weight Selection
- Only Use 3 Folds

In [1]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

def log_mean_absolute_error(y_true, y_pred):
    return np.log(mean_absolute_error(y_true, y_pred))
%matplotlib inline

In [ ]:
types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC','3JHH','3JHC', '3JHN']

def get_sub_oof(t):
    """
    Grab the sub and oof files for a give type. 
    """
    oof = []
    sub = []
    model_ids = []
    for file in sorted(os.listdir(f'../type_results/{t}/')):
        if ('sub' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            model_ids.append(model_id)
            df = pd.read_parquet(f'../type_results/{t}/{file}')
            if len(sub) == 0:
                sub = df.rename(columns={'scalar_coupling_constant': model_id})
            else:
                sub[model_id] = df['scalar_coupling_constant']
        elif ('oof' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            #print(model_id)
            df = pd.read_parquet(f'../type_results/{t}/{file}')
            if len(oof) == 0:
                oof = df.rename(columns={'oof_preds': model_id})
            else:
                oof[model_id] = df['oof_preds']
    sub = sub.loc[sub['type'] == t]
    oof = oof.loc[oof['type'] == t]
    return sub, oof, model_ids

def get_best_weights(oof, l5, t):
    if len(l5) == 1:
        return [1]
    lmae_best = 0
    for x in tqdm(range(0, 5000)):
        ws = []
        for x in range(0, len(l5)):
            if x == len(l5)-1:
                ws.append(random.randint(1, 50))
            else:
                ws.append(random.randint(0, 50))
        idx = 0
        blend = np.zeros(len(oof[l5[0]]))
        for w in ws:
            blend += w * oof[l5[idx]]
            idx += 1
        blend = blend / np.sum(ws)
        lmae = log_mean_absolute_error(oof['scalar_coupling_constant'],
                            blend)
        if lmae < lmae_best:
            lmae_best = lmae
            bws = ws
    print(f'{t}: Best score {lmae:0.5f} - with weights {bws}')
    return bws # Return the best weights

def create_best_blends():
    oofs = []
    subs = []
    for t in types:
        print(f'{t}: Running for type {t}')
        tsub, toof, tmodel_ids = get_sub_oof(t)
        print(f'{t}: Has model names {tmodel_ids}')
        l5 = tmodel_ids[-5:] # Last 5 models
        bws = get_best_weights(toof, l5, t)
        tot_weight = np.sum(bws)
        tsub['scalar_coupling_constant'] = 0
        toof['oof_blend'] = 0
        idx = 0
        for w in bws:
            tsub['scalar_coupling_constant'] += (w * tsub[l5[idx]])
            toof['oof_blend'] += (w * toof[l5[idx]])
            idx += 1
        tsub['scalar_coupling_constant'] /= tot_weight
        toof['oof_blend'] /= tot_weight
        tsub = tsub[['id','molecule_name','type','scalar_coupling_constant']].copy()
        toof = toof[['id','type','scalar_coupling_constant','oof_blend']].copy()
        subs.append(tsub)
        oofs.append(toof)
    final_oof = pd.concat(oofs).sort_values('id').reset_index(drop=True)
    final_sub = pd.concat(subs).sort_values('id').reset_index(drop=True)
    return final_oof, final_sub
    
final_oof, final_sub = create_best_blends()

1JHC: Running for type 1JHC


  0%|          | 3/5000 [00:00<03:32, 23.56it/s]

1JHC: Has model names ['M047-0.7977', 'M048-0.8258']


 22%|██▏       | 1091/5000 [00:34<02:01, 32.15it/s]

# Score

In [18]:
for i, d in final_oof.groupby('type'):
    print(i, '{:0.5f}'.format(mean_absolute_error(d['scalar_coupling_constant'], d['oof_blend'])))
glmae = group_mean_log_mae(final_oof['scalar_coupling_constant'], final_oof['oof_blend'], final_oof['type'])
print('\nGroup LMAE')
print('{:0.5f}'.format(glmae))

1JHC 0.40569
1JHN 0.23375
2JHC 0.16186
2JHH 0.08987
2JHN 0.09972
3JHC 0.19953
3JHH 0.10608
3JHN 0.08749

Group LMAE
-1.89788


# Validate Sub

In [10]:
sub_good = pd.read_csv('../submissions/BLEND014_sub_-1.85550CV.csv')
final_sub['good_scc'] = sub_good['scalar_coupling_constant']
final_sub[['scalar_coupling_constant','good_scc']].corr()

,scalar_coupling_constant,good_scc
scalar_coupling_constant,1.000000,0.999996
good_scc,0.999996,1.000000


In [28]:
%%javascript
IPython.notebook.kernel.execute(`notebookName = '${window.document.getElementById("notebook_name").innerHTML}'`);

<IPython.core.display.Javascript object>

# Create Submission and OOF Files

In [34]:
import json
import os.path
import re
import ipykernel
import requests

#try:  # Python 3
#    from urllib.parse import urljoin
#except ImportError:  # Python 2
#    from urlparse import urljoin

# Alternative that works for both Python 2 and 3:
from requests.compat import urljoin

try:  # Python 3 (see Edit2 below for why this may not work in Python 2)
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers


def get_notebook_name():
    """
    Return the full path of the jupyter notebook.
    """
    kernel_id = re.search('kernel-(.*).json',
                          ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

In [39]:
BLEND_NUMBER = get_notebook_name().split('/')[-1].replace('.ipynb','').replace('-','')
print(f'NAME TO SAVE {BLEND_NUMBER}')

NAME TO SAVE BLEND018


In [41]:
# Save the Results
final_sub[['id','scalar_coupling_constant']].to_csv(f'../submissions/{BLEND_NUMBER}_sub_{glmae:0.5f}CV.csv', index=False)
final_oof.to_csv(f'../oof/{BLEND_NUMBER}_oof_{glmae:0.5f}CV.csv', index=False)